# Enhanced CNN Light Curve Model for NASA Exoplanet Classification

## 🚀 Welcome to Exoplanet Detection with Deep Learning!

This notebook demonstrates how to use **Convolutional Neural Networks (CNNs)** to detect exoplanets from light curve data. Whether you're new to astronomy, machine learning, or both, this notebook will guide you through the entire process step by step.

### What You'll Learn:
1. **What are exoplanets and how do we detect them?**
2. **Understanding light curves and transit photometry**
3. **How CNNs can analyze time-series astronomical data**
4. **Building and training sophisticated neural network architectures**
5. **Evaluating model performance for scientific applications**

### Our Goal:
Achieve **80%+ F1 score** in classifying astronomical objects using deep learning on synthetic light curves.

---

## 🌟 Background: What Are Exoplanets?

**Exoplanets** (extrasolar planets) are planets that orbit stars outside our solar system. Since 1995, astronomers have discovered over 5,000 exoplanets using various detection methods.

### The Transit Method

The **transit method** is one of the most successful techniques for finding exoplanets. Here's how it works:

1. **A planet orbits in front of its host star** (from our perspective)
2. **The planet blocks a tiny amount of starlight** during transit
3. **We measure this periodic dimming** in the star's brightness
4. **The pattern tells us about the planet's size and orbit**

![Transit Method Illustration](https://upload.wikimedia.org/wikipedia/commons/thumb/1/10/Transiting_exoplanet.jpg/400px-Transiting_exoplanet.jpg)

### Light Curves

A **light curve** is a graph showing how a star's brightness changes over time. For exoplanet detection:
- **X-axis**: Time (usually in days or orbital phase)
- **Y-axis**: Relative brightness (normalized flux)
- **Transit signature**: Periodic dips in brightness

### Why Use Machine Learning?

Traditional methods struggle with:
- **Noise** in the data from instruments and stellar variability
- **False positives** from eclipsing binaries and other phenomena
- **Weak signals** from small planets

**Convolutional Neural Networks** can:
- Learn complex patterns in light curves
- Distinguish real transits from false positives
- Handle noisy, real-world data effectively

---

## 📚 Part 1: Setting Up Our Environment

Let's start by importing all the libraries we'll need. Don't worry if you're not familiar with all of them - we'll explain each one as we use it!

In [1]:
# Standard data science libraries
import pandas as pd           # For handling tabular data (like spreadsheets)
import numpy as np            # For numerical computations and arrays
import matplotlib.pyplot as plt  # For creating plots and visualizations

# Machine learning libraries
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

# Scientific computing libraries
from scipy import stats, signal    # For statistical functions and signal processing
from scipy.interpolate import interp1d  # For interpolating data points

# Utility libraries
import warnings
import time
import pickle
from pathlib import Path
import sys

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("✅ Standard libraries imported successfully!")

✅ Standard libraries imported successfully!


In [2]:
# Add our dataset loader to the Python path
sys.path.append('/Users/kkgogada/Code/NASASAC2025')
from dataset.loader import KeplerDatasetLoader

print("✅ Dataset loader imported successfully!")

✅ Dataset loader imported successfully!


In [2]:
# Deep learning libraries (TensorFlow/Keras)
# TensorFlow is Google's machine learning framework
# Keras is a high-level API that makes building neural networks easier

# Import numpy first for random seed setting
import numpy as np

try:
    import tensorflow as tf
    keras = tf.keras
    from tensorflow.keras import layers, models, callbacks, optimizers
    from tensorflow.keras.utils import to_categorical
    TENSORFLOW_AVAILABLE = True
    print("✅ TensorFlow available")
    print(f"   TensorFlow version: {tf.__version__}")
    
    # Set random seeds for reproducible results
    np.random.seed(42)
    tf.random.set_seed(42)
    print("✅ Random seeds set for reproducibility")
    
except ImportError as e:
    print(f"❌ TensorFlow not available: {e}")
    print("   Please install TensorFlow: pip install tensorflow")
    TENSORFLOW_AVAILABLE = False

✅ TensorFlow available
   TensorFlow version: 2.16.2
✅ Random seeds set for reproducibility


---

## 🔬 Part 2: Understanding Synthetic Light Curve Generation

Before we can train a neural network, we need data! While real exoplanet data exists, we'll create **synthetic light curves** to:

1. **Control the parameters** we want to test
2. **Generate large amounts of training data**
3. **Understand the physics** behind exoplanet transits

### The Physics Behind Transits

When a planet passes in front of its star:
- The **depth** of the transit depends on the planet's size
- The **duration** depends on the planet's orbital speed and size
- The **shape** depends on the star's limb darkening and planet's path

Let's build a class that can generate realistic synthetic light curves!

In [3]:
class SyntheticLightCurveGenerator:
    """
    Generate realistic synthetic phase-folded light curves from transit parameters.
    
    This class simulates what happens when a planet transits (passes in front of)
    its host star, creating the characteristic dip in brightness that we observe.
    """
    
    def __init__(self, n_phase_samples=2001):
        """
        Initialize the light curve generator.
        
        Parameters:
        -----------
        n_phase_samples : int
            Number of data points in the light curve (default: 2001)
            More points = higher resolution but more computation
        """
        self.n_phase_samples = n_phase_samples
        # Create phase array from -0.5 to +0.5 (one complete orbit)
        # Phase 0.0 = center of transit
        self.phase = np.linspace(-0.5, 0.5, n_phase_samples)
        
    def mandel_agol_transit(self, phase, depth, duration, impact_param=0.5):
        """
        Generate primary transit using simplified Mandel-Agol model.
        
        This is based on the mathematical model that describes how light
        is blocked when a planet passes in front of a star.
        
        Parameters:
        -----------
        phase : array
            Orbital phase values (-0.5 to +0.5)
        depth : float
            Maximum depth of transit (0.001 = 0.1% decrease in brightness)
        duration : float
            Duration of transit in phase units (0.1 = 10% of orbital period)
        impact_param : float
            How close planet passes to star center (0=center, 1=edge)
        
        Returns:
        --------
        flux : array
            Relative brightness values (1.0 = normal brightness)
        """
        # Start with constant brightness (no transit)
        flux = np.ones(len(phase))
        
        # Convert duration to phase units
        half_duration = duration / 2
        
        # Find which phase points are during transit
        in_transit = np.abs(phase) <= half_duration
        
        if np.any(in_transit):
            # Simplified limb-darkened transit
            # Real stars are brighter in the center, dimmer at edges
            phase_norm = phase[in_transit] / half_duration
            
            # Create U-shaped transit with limb darkening effect
            for i, p_norm in enumerate(phase_norm):
                # Calculate distance from star center
                r = np.sqrt(p_norm**2 + impact_param**2)
                
                if r <= 1.0:  # Planet is in front of star
                    # Calculate limb darkening (stars dimmer at edges)
                    mu = np.sqrt(1 - r**2) if r < 1 else 0
                    # Quadratic limb darkening law (common in astronomy)
                    limb_dark = 1 - 0.3 * (1 - mu) - 0.1 * (1 - mu)**2
                    # Apply transit depth with limb darkening
                    flux[in_transit][i] = 1 - depth * limb_dark
        
        return flux

print("✅ SyntheticLightCurveGenerator class defined!")
print("   This class can simulate realistic exoplanet transits.")

✅ SyntheticLightCurveGenerator class defined!
   This class can simulate realistic exoplanet transits.


In [4]:
# Continue with more methods for the SyntheticLightCurveGenerator class

def add_stellar_variability(self, flux, amplitude=0.001):
    """
    Add realistic stellar variability to the light curve.
    
    Real stars aren't perfectly constant - they have:
    - Rotation (star spots cause periodic brightness changes)
    - Pulsations (some stars naturally pulsate)
    - Granulation (convection cells on surface)
    
    Parameters:
    -----------
    flux : array
        Input light curve
    amplitude : float
        Strength of stellar variability (default: 0.001 = 0.1%)
    
    Returns:
    --------
    modified_flux : array
        Light curve with added stellar variability
    """
    # Create smooth variability using multiple sine waves
    # (like combining different rotation periods and pulsation modes)
    variability = 0
    
    for i in range(3):  # Combine 3 different variability sources
        freq = np.random.uniform(0.5, 3.0)  # Different frequencies
        phase_offset = np.random.uniform(0, 2*np.pi)  # Random phase
        amp = amplitude * np.random.uniform(0.3, 1.0)  # Different amplitudes
        
        # Add sinusoidal variability
        variability += amp * np.sin(2 * np.pi * freq * self.phase + phase_offset)
    
    return flux + variability

def add_noise(self, flux, snr=100):
    """
    Add realistic photometric noise to simulate real observations.
    
    Real astronomical data always has noise from:
    - Photon noise (fundamental quantum limit)
    - Instrument noise (detector imperfections)
    - Atmospheric noise (for ground-based observations)
    
    Parameters:
    -----------
    flux : array
        Input light curve
    snr : float
        Signal-to-noise ratio (higher = less noisy)
        100 = good space-based data, 10 = challenging ground-based data
    
    Returns:
    --------
    noisy_flux : array
        Light curve with added realistic noise
    """
    if snr <= 0:
        return flux
        
    # Calculate noise level from signal-to-noise ratio
    signal_depth = np.max(flux) - np.min(flux)
    noise_level = signal_depth / snr if signal_depth > 0 else 1e-6
    
    # Add white noise (random, uncorrelated)
    white_noise = np.random.normal(0, noise_level, len(flux))
    
    # Add small amount of red noise (correlated, systematic)
    red_noise_amp = noise_level * 0.3
    red_noise = np.cumsum(np.random.normal(0, red_noise_amp, len(flux)))
    red_noise = red_noise - np.mean(red_noise)  # Remove overall trend
    red_noise = red_noise / np.std(red_noise) * red_noise_amp  # Normalize
    
    return flux + white_noise + red_noise

# Add these methods to our SyntheticLightCurveGenerator class
SyntheticLightCurveGenerator.add_stellar_variability = add_stellar_variability
SyntheticLightCurveGenerator.add_noise = add_noise

print("✅ Added stellar variability and noise methods!")
print("   Now we can simulate realistic observational conditions.")

✅ Added stellar variability and noise methods!
   Now we can simulate realistic observational conditions.


In [5]:
def generate_light_curve(self, transit_params, add_variability=True, add_secondary=False):
    """
    Generate complete synthetic light curve with all effects.
    
    This is our main function that combines:
    1. Basic transit shape
    2. Stellar variability
    3. Secondary eclipses (for some systems)
    4. Realistic noise
    
    Parameters:
    -----------
    transit_params : dict
        Dictionary with transit parameters:
        - 'depth': Transit depth (0.001 = 0.1%)
        - 'duration': Transit duration (0.1 = 10% of orbit)
        - 'impact_param': Impact parameter (0-1)
        - 'snr': Signal-to-noise ratio
    add_variability : bool
        Whether to add stellar variability
    add_secondary : bool
        Whether to potentially add secondary eclipse
    
    Returns:
    --------
    flux : array
        Complete synthetic light curve
    """
    # Extract parameters with default values
    depth = transit_params.get('depth', 0.001)
    duration = transit_params.get('duration', 0.1)
    impact_param = transit_params.get('impact_param', 0.5)
    snr = transit_params.get('snr', 100)
    
    # Start with flat light curve (constant brightness)
    flux = np.ones(self.n_phase_samples)
    
    # Add primary transit (the main exoplanet signal)
    flux = self.mandel_agol_transit(self.phase, depth, duration, impact_param)
    
    # Add stellar variability (makes it more realistic)
    if add_variability:
        variability_amp = np.random.uniform(0.0005, 0.002)  # Random strength
        flux = self.add_stellar_variability(flux, variability_amp)
    
    # Add secondary eclipse (rare, when planet goes behind star)
    if add_secondary and np.random.random() < 0.2:  # Only 20% chance
        secondary_depth = depth * np.random.uniform(0.001, 0.01)  # Much shallower
        secondary_phase = 0.5 + np.random.uniform(-0.05, 0.05)  # Opposite side
        secondary_mask = np.abs(self.phase - secondary_phase) <= 0.02
        flux[secondary_mask] += secondary_depth  # Adds brightness (thermal emission)
    
    # Add realistic noise
    if snr > 0:
        flux = self.add_noise(flux, snr)
        
    return flux

# Add this method to our class
SyntheticLightCurveGenerator.generate_light_curve = generate_light_curve

print("✅ Complete light curve generation method added!")
print("   Ready to create realistic synthetic exoplanet data.")

✅ Complete light curve generation method added!
   Ready to create realistic synthetic exoplanet data.


### 🧪 Let's Test Our Light Curve Generator!

Before moving on, let's create some example light curves to see what our synthetic data looks like:

In [6]:
# Create a light curve generator
generator = SyntheticLightCurveGenerator(n_phase_samples=2001)

# Define some example transit parameters
example_params = {
    'depth': 0.01,      # 1% transit depth (relatively large planet)
    'duration': 0.15,   # 15% of orbital period
    'impact_param': 0.3, # Passes close to star center
    'snr': 50           # Moderate noise level
}

# Generate example light curves
print("Generating example light curves...")

# Perfect transit (no noise or variability)
perfect_transit = generator.mandel_agol_transit(
    generator.phase, 
    example_params['depth'], 
    example_params['duration'], 
    example_params['impact_param']
)

# Realistic light curve (with all effects)
realistic_curve = generator.generate_light_curve(example_params, add_variability=True)

print("✅ Example light curves generated!")
print(f"   Transit depth: {example_params['depth']*100:.1f}%")
print(f"   Duration: {example_params['duration']*100:.1f}% of orbit")
print(f"   Signal-to-noise ratio: {example_params['snr']}")

Generating example light curves...
✅ Example light curves generated!
   Transit depth: 1.0%
   Duration: 15.0% of orbit
   Signal-to-noise ratio: 50


In [ ]:
# Import matplotlib for plotting (in case it wasn't imported earlier)
import matplotlib.pyplot as plt

# Plot our example light curves
plt.figure(figsize=(15, 8))

# Plot 1: Perfect transit
plt.subplot(2, 2, 1)
plt.plot(generator.phase, perfect_transit, 'b-', linewidth=2)
plt.xlabel('Orbital Phase')
plt.ylabel('Relative Flux')
plt.title('Perfect Transit (No Noise or Variability)')
plt.grid(True, alpha=0.3)

# Plot 2: Realistic light curve
plt.subplot(2, 2, 2)
plt.plot(generator.phase, realistic_curve, 'r-', linewidth=1, alpha=0.7)
plt.xlabel('Orbital Phase')
plt.ylabel('Relative Flux')
plt.title('Realistic Light Curve (With Noise & Variability)')
plt.grid(True, alpha=0.3)

# Plot 3: Zoom in on transit
plt.subplot(2, 2, 3)
transit_mask = np.abs(generator.phase) <= 0.2  # Focus on transit region
plt.plot(generator.phase[transit_mask], perfect_transit[transit_mask], 'b-', linewidth=2, label='Perfect')
plt.plot(generator.phase[transit_mask], realistic_curve[transit_mask], 'r-', linewidth=1, alpha=0.7, label='Realistic')
plt.xlabel('Orbital Phase')
plt.ylabel('Relative Flux')
plt.title('Transit Zoom-In')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 4: Multiple random examples
plt.subplot(2, 2, 4)
colors = ['red', 'blue', 'green', 'orange', 'purple']
for i in range(5):
    # Generate random parameters
    random_params = {
        'depth': np.random.uniform(0.001, 0.02),
        'duration': np.random.uniform(0.05, 0.25),
        'impact_param': np.random.uniform(0.0, 0.8),
        'snr': np.random.uniform(20, 100)
    }
    
    curve = generator.generate_light_curve(random_params)
    plt.plot(generator.phase, curve, color=colors[i], alpha=0.6, linewidth=1)

plt.xlabel('Orbital Phase')
plt.ylabel('Relative Flux')
plt.title('Random Synthetic Light Curves')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📈 Light curve examples plotted!")
print("   Notice how the realistic curves have noise and variability,")
print("   making them more challenging but representative of real data.")

NameError: name 'plt' is not defined

---

## 🧠 Part 3: Understanding Convolutional Neural Networks (CNNs)

Now that we can generate synthetic data, let's understand **why** and **how** CNNs work for this problem.

### What are CNNs?

**Convolutional Neural Networks** are a type of deep learning model originally designed for image analysis, but they work excellently for time-series data like light curves because:

1. **Local pattern detection**: They can identify the characteristic "U" shape of transits
2. **Translation invariance**: They can find transits anywhere in the light curve
3. **Hierarchical features**: Lower layers detect edges, higher layers detect complex patterns
4. **Parameter sharing**: The same filters are applied across the entire time series

### Our CNN Architecture Strategy

We'll implement **two complementary approaches**:

1. **Single-View CNN**: Analyzes the entire light curve at once
2. **Two-View CNN**: Combines global and local (transit-focused) views

### Key CNN Components We'll Use:

- **Conv1D layers**: Detect patterns in time-series data
- **Residual connections**: Help with training deep networks
- **Attention mechanisms**: Focus on important parts of the data
- **Batch normalization**: Stabilizes training
- **Dropout**: Prevents overfitting

Let's build our CNN pipeline!

In [ ]:
class CNNLightCurvePipeline:
    """
    Enhanced CNN pipeline for light curve classification.
    
    This class handles the complete machine learning pipeline:
    1. Data preparation and synthetic light curve generation
    2. Advanced preprocessing for CNNs
    3. Model creation (single-view and two-view architectures)
    4. Training with cross-validation
    5. Evaluation and results reporting
    """
    
    def __init__(self, random_state=42):
        """
        Initialize the CNN pipeline.
        
        Parameters:
        -----------
        random_state : int
            Random seed for reproducible results
        """
        self.random_state = random_state
        self.lc_generator = SyntheticLightCurveGenerator(n_phase_samples=2001)
        self.models = {}      # Store trained models
        self.results = {}     # Store evaluation results
        
        print("🚀 CNN Light Curve Pipeline initialized!")
        print(f"   Random state: {random_state}")
        print(f"   Light curve resolution: {self.lc_generator.n_phase_samples} points")

# Create our pipeline instance
pipeline = CNNLightCurvePipeline(random_state=42)
print("✅ Pipeline ready for use!")

### 📊 Part 4: Data Preparation

Our next step is preparing the data for training. This involves:

1. **Loading tabular data** (planet parameters from catalogs)
2. **Generating synthetic light curves** for each set of parameters
3. **Quality control** to ensure valid data
4. **Creating training/validation splits**

In [ ]:
def prepare_data(self, n_samples=2000):
    """
    Load tabular data and generate synthetic light curves.
    
    This function:
    1. Loads real exoplanet parameters from catalogs
    2. Generates synthetic light curves based on those parameters
    3. Performs quality control and validation
    4. Returns clean, ready-to-use datasets
    
    Parameters:
    -----------
    n_samples : int
        Number of samples to use (default: 2000)
        More samples = better training but longer time
    
    Returns:
    --------
    X_lightcurves : array
        Generated light curves (n_samples, 2001)
    final_tabular : DataFrame
        Corresponding tabular features
    final_y : array
        Classification labels (0=False Positive, 1=Planet Candidate, 2=Confirmed Planet)
    final_groups : array
        Grouping information for cross-validation
    """
    print("📊 Loading Dataset and Generating Light Curves...")
    
    # Load real exoplanet data from catalogs (Kepler, TESS, etc.)
    print("   Loading real exoplanet catalog data...")
    loader = KeplerDatasetLoader()
    X_tabular, y, groups, feature_names = loader.load_dataset()
    
    # Use subset for development (you can increase this for production)
    n_samples = min(n_samples, len(X_tabular))
    indices = np.random.choice(len(X_tabular), n_samples, replace=False)
    
    X_tabular_subset = X_tabular.iloc[indices]
    y_subset = y[indices]
    groups_subset = groups.iloc[indices]
    
    print(f"   ✓ Using {n_samples} samples for CNN training")
    print(f"   ✓ Class distribution: {dict(pd.Series(y_subset).value_counts().sort_index())}")
    
    # Generate synthetic light curves based on real parameters
    light_curves = []
    valid_indices = []
    
    print("   🔧 Generating synthetic light curves...")
    for i, (idx, row) in enumerate(X_tabular_subset.iterrows()):
        # Progress reporting
        if i % 200 == 0:
            print(f"      Progress: {i}/{len(X_tabular_subset)} ({i/len(X_tabular_subset)*100:.1f}%)")
        
        try:
            # Extract transit parameters from catalog data
            # If parameters are missing, use reasonable random values
            transit_params = {
                'depth': row.get('depth', np.random.uniform(0.0001, 0.01)),
                'duration': row.get('duration', np.random.uniform(0.02, 0.2)),
                'impact_param': row.get('impact_param', np.random.uniform(0.0, 0.9)),
                'snr': row.get('snr', np.random.uniform(10, 200))
            }
            
            # Ensure parameters are within reasonable physical ranges
            param_ranges = {
                'depth': (0.0001, 0.1),      # 0.01% to 10% (Earth to Jupiter-sized)
                'duration': (0.01, 0.5),     # 1% to 50% of orbital period
                'impact_param': (0.0, 1.0),  # 0 (center) to 1 (edge)
                'snr': (5, 1000)             # Very noisy to very clean
            }
            
            for key, (min_val, max_val) in param_ranges.items():
                transit_params[key] = np.clip(transit_params[key], min_val, max_val)
            
            # Generate synthetic light curve
            flux = self.lc_generator.generate_light_curve(
                transit_params, 
                add_variability=True, 
                add_secondary=True
            )
            
            light_curves.append(flux)
            valid_indices.append(i)
            
        except Exception as e:
            print(f"      Warning: Failed for sample {i}: {e}")
            continue
    
    # Convert to arrays for machine learning
    X_lightcurves = np.array(light_curves)
    valid_indices = np.array(valid_indices)
    
    # Update corresponding tabular data
    final_tabular = X_tabular_subset.iloc[valid_indices]
    final_y = y_subset[valid_indices]
    final_groups = groups_subset.iloc[valid_indices]
    
    print(f"   ✅ Generated {len(X_lightcurves)} light curves successfully")
    print(f"   ✅ Success rate: {len(X_lightcurves)/n_samples*100:.1f}%")
    
    return X_lightcurves, final_tabular, final_y, final_groups

# Add this method to our pipeline class
CNNLightCurvePipeline.prepare_data = prepare_data

print("✅ Data preparation method ready!")
print("   This will load real exoplanet parameters and generate synthetic light curves.")

### 🔧 Part 5: Advanced Preprocessing for CNNs

Before feeding data to our neural networks, we need to preprocess it carefully:

1. **Detrending**: Remove systematic trends that aren't related to planets
2. **Normalization**: Ensure consistent scaling across all light curves
3. **Two-view creation**: Generate global and local (transit-focused) views
4. **Data formatting**: Shape arrays correctly for CNN input

This preprocessing is crucial for CNN performance!

In [ ]:
def preprocess_lightcurves(self, light_curves, create_two_views=True):
    """
    Advanced preprocessing for CNN input.
    
    This function applies several important preprocessing steps:
    1. Detrending: Remove long-term trends not related to transits
    2. Normalization: Scale data consistently
    3. Two-view creation: Global + local transit views
    4. Proper formatting for CNN layers
    
    Parameters:
    -----------
    light_curves : array
        Input light curves (n_samples, n_phase_points)
    create_two_views : bool
        Whether to create separate global and local views
    
    Returns:
    --------
    If create_two_views=True:
        global_views, local_views : tuple of arrays
    If create_two_views=False:
        processed : array
            Preprocessed light curves ready for single-view CNN
    """
    print("🔧 Preprocessing Light Curves for CNN...")
    
    processed = light_curves.copy()
    
    # Step 1: Detrending
    print("   📉 Detrending light curves...")
    for i in range(len(processed)):
        phase = self.lc_generator.phase
        
        # Polynomial detrending using out-of-transit regions
        # We don't want to remove the transit signal itself!
        out_of_transit = np.abs(phase) >= 0.25  # Use data far from transit
        
        if np.any(out_of_transit):
            # Fit 2nd-degree polynomial to out-of-transit data
            p = np.polyfit(phase[out_of_transit], processed[i][out_of_transit], 2)
            trend = np.polyval(p, phase)
            # Remove trend but preserve median level
            processed[i] = processed[i] - trend + np.median(processed[i])
    
    # Step 2: Robust normalization
    print("   📏 Normalizing light curves...")
    for i in range(len(processed)):
        # Use data far from transit for baseline estimation
        far_from_transit = np.abs(self.lc_generator.phase) >= 0.35
        
        if np.any(far_from_transit):
            # Use robust statistics (median and MAD instead of mean and std)
            baseline_median = np.median(processed[i][far_from_transit])
            baseline_mad = np.median(np.abs(processed[i][far_from_transit] - baseline_median))
            
            # Normalize using Median Absolute Deviation (more robust to outliers)
            processed[i] = (processed[i] - baseline_median) / (1.4826 * baseline_mad + 1e-8) + 1.0
        else:
            # Fallback normalization
            processed[i] = processed[i] / (np.median(processed[i]) + 1e-8)
    
    if not create_two_views:
        # Single view: return full light curves reshaped for CNN
        return processed.reshape(processed.shape[0], processed.shape[1], 1)
    
    # Step 3: Create two complementary views
    print("   👀 Creating global and local views...")
    
    # Global view: entire light curve (captures long-term patterns)
    global_views = processed
    
    # Local view: zoomed-in transit region (captures fine details)
    phase = self.lc_generator.phase
    transit_center_idx = len(phase) // 2  # Phase = 0.0
    zoom_half_width = 250  # ±250 points around transit
    
    local_views = []
    for i in range(len(processed)):
        # Extract transit region
        start_idx = max(0, transit_center_idx - zoom_half_width)
        end_idx = min(len(phase), transit_center_idx + zoom_half_width + 1)
        
        local_curve = processed[i][start_idx:end_idx]
        
        # Ensure exactly 501 samples for consistent CNN input
        if len(local_curve) < 501:
            # Pad with edge values if too short
            pad_left = (501 - len(local_curve)) // 2
            pad_right = 501 - len(local_curve) - pad_left
            local_curve = np.pad(local_curve, (pad_left, pad_right), mode='edge')
        elif len(local_curve) > 501:
            # Trim if too long
            excess = len(local_curve) - 501
            trim_left = excess // 2
            local_curve = local_curve[trim_left:trim_left + 501]
        
        local_views.append(local_curve)
    
    local_views = np.array(local_views)
    
    # Step 4: Reshape for CNN input (add channel dimension)
    global_views = global_views.reshape(global_views.shape[0], global_views.shape[1], 1)
    local_views = local_views.reshape(local_views.shape[0], local_views.shape[1], 1)
    
    print(f"   ✅ Preprocessed - Global: {global_views.shape}, Local: {local_views.shape}")
    
    return global_views, local_views

# Add preprocessing method to our pipeline
CNNLightCurvePipeline.preprocess_lightcurves = preprocess_lightcurves

print("✅ Preprocessing pipeline ready!")
print("   This includes detrending, normalization, and two-view creation.")

---

## 🏗️ Part 6: Building CNN Architectures

Now we'll create two different CNN architectures to compare their performance:

1. **Two-View CNN**: Uses both global (full light curve) and local (transit zoom) views
2. **Single-View CNN**: Uses only the full light curve

### Why Two Views?

- **Global view** captures long-term stellar variability and orbital patterns
- **Local view** focuses on fine transit details and shape
- **Attention mechanism** learns to weight the importance of each view
- **Ensemble effect** often performs better than either view alone

Let's build these architectures!

In [ ]:
def create_two_view_cnn(self, global_length=2001, local_length=501, num_classes=3):
    """
    Create sophisticated two-view CNN architecture.
    
    This architecture combines:
    1. Global branch: Processes entire light curve (2001 points)
    2. Local branch: Processes transit-focused region (501 points) 
    3. Attention mechanism: Learns optimal weighting of both views
    4. Residual connections: Helps with training deeper networks
    
    Architecture Details:
    - Multiple Conv1D layers with increasing filters
    - Batch normalization for training stability
    - Residual skip connections for better gradient flow
    - Attention-based fusion of global and local features
    - Progressive dropout for regularization
    
    Parameters:
    -----------
    global_length : int
        Length of global view (default: 2001)
    local_length : int  
        Length of local view (default: 501)
    num_classes : int
        Number of output classes (default: 3)
        
    Returns:
    --------
    model : keras.Model
        Compiled two-view CNN model
    """
    if not TENSORFLOW_AVAILABLE:
        print("❌ TensorFlow not available - cannot create CNN")
        return None
        
    print("   🏗️ Building Two-View CNN Architecture...")
    
    # === GLOBAL VIEW BRANCH ===
    # Processes the entire light curve to capture broad patterns
    global_input = keras.Input(shape=(global_length, 1), name='global_view')
    
    # First global conv block with residual connection
    g1 = layers.Conv1D(64, 51, activation='relu', padding='same', name='global_conv1a')(global_input)
    g1 = layers.BatchNormalization()(g1)
    g1 = layers.Conv1D(64, 51, activation='relu', padding='same', name='global_conv1b')(g1)
    g1 = layers.BatchNormalization()(g1)
    # Residual connection (helps with training deep networks)
    g1_skip = layers.Conv1D(64, 1, padding='same', name='global_skip1')(global_input)
    g1 = layers.Add()([g1, g1_skip])
    g1 = layers.MaxPooling1D(4)(g1)  # Reduce resolution
    g1 = layers.Dropout(0.1)(g1)
    
    # Second global conv block
    g2 = layers.Conv1D(128, 25, activation='relu', padding='same', name='global_conv2a')(g1)
    g2 = layers.BatchNormalization()(g2)
    g2 = layers.Conv1D(128, 25, activation='relu', padding='same', name='global_conv2b')(g2)
    g2 = layers.BatchNormalization()(g2)
    g2_skip = layers.Conv1D(128, 1, padding='same', name='global_skip2')(g1)
    g2 = layers.Add()([g2, g2_skip])
    g2 = layers.MaxPooling1D(4)(g2)
    g2 = layers.Dropout(0.15)(g2)
    
    # Final global conv block
    g3 = layers.Conv1D(256, 11, activation='relu', padding='same', name='global_conv3')(g2)
    g3 = layers.BatchNormalization()(g3)
    g3 = layers.GlobalAveragePooling1D()(g3)  # Convert to fixed-size feature vector
    
    # === LOCAL VIEW BRANCH ===
    # Processes zoomed transit region for fine details
    local_input = keras.Input(shape=(local_length, 1), name='local_view')
    
    # First local conv block
    l1 = layers.Conv1D(96, 7, activation='relu', padding='same', name='local_conv1a')(local_input)
    l1 = layers.BatchNormalization()(l1)
    l1 = layers.Conv1D(96, 7, activation='relu', padding='same', name='local_conv1b')(l1)
    l1 = layers.BatchNormalization()(l1)
    l1_skip = layers.Conv1D(96, 1, padding='same', name='local_skip1')(local_input)
    l1 = layers.Add()([l1, l1_skip])
    l1 = layers.MaxPooling1D(2)(l1)
    l1 = layers.Dropout(0.1)(l1)
    
    # Second local conv block
    l2 = layers.Conv1D(192, 5, activation='relu', padding='same', name='local_conv2')(l1)
    l2 = layers.BatchNormalization()(l2)
    l2_skip = layers.Conv1D(192, 1, padding='same', name='local_skip2')(l1)
    l2 = layers.Add()([l2, l2_skip])
    l2 = layers.MaxPooling1D(2)(l2)
    l2 = layers.Dropout(0.15)(l2)
    
    # Final local conv block
    l3 = layers.Conv1D(384, 3, activation='relu', padding='same', name='local_conv3')(l2)
    l3 = layers.BatchNormalization()(l3)
    l3 = layers.GlobalAveragePooling1D()(l3)
    
    # === ATTENTION MECHANISM ===
    # Learn optimal weighting of global vs local features
    print("      Adding attention mechanism...")
    attention_input = layers.concatenate([g3, l3], name='attention_concat')
    attention_dense = layers.Dense(256, activation='relu', name='attention_dense')(attention_input)
    attention_weights = layers.Dense(2, activation='softmax', name='attention_weights')(attention_dense)
    
    # Apply attention weights
    global_weighted = layers.Multiply(name='global_attention')([g3, attention_weights[:, 0:1]])
    local_weighted = layers.Multiply(name='local_attention')([l3, attention_weights[:, 1:2]])
    
    # === FEATURE FUSION ===
    fused = layers.concatenate([global_weighted, local_weighted], name='feature_fusion')
    
    # === CLASSIFICATION HEAD ===
    x = layers.Dense(512, activation='relu', name='dense1')(fused)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    
    x = layers.Dense(256, activation='relu', name='dense2')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    x = layers.Dense(128, activation='relu', name='dense3')(x)
    x = layers.Dropout(0.2)(x)
    
    outputs = layers.Dense(num_classes, activation='softmax', name='classification')(x)
    
    # Create and compile model
    model = keras.Model(inputs=[global_input, local_input], outputs=outputs, name='TwoViewCNN')
    
    print(f"      ✅ Two-View CNN created: {model.count_params():,} parameters")
    
    return model

# Add method to pipeline
CNNLightCurvePipeline.create_two_view_cnn = create_two_view_cnn

print("✅ Two-View CNN architecture defined!")

In [ ]:
def create_single_view_cnn(self, input_length=2001, num_classes=3):
    """
    Create enhanced single-view CNN for comparison.
    
    This is our baseline architecture that processes only the full light curve.
    It's simpler than the two-view model but still uses modern techniques:
    - Residual connections for better training
    - Batch normalization for stability  
    - Progressive dropout for regularization
    - Global average pooling to reduce parameters
    
    Parameters:
    -----------
    input_length : int
        Length of input light curve (default: 2001)
    num_classes : int
        Number of output classes (default: 3)
        
    Returns:
    --------
    model : keras.Model
        Compiled single-view CNN model
    """
    if not TENSORFLOW_AVAILABLE:
        print("❌ TensorFlow not available - cannot create CNN")
        return None
        
    print("   🏗️ Building Single-View CNN Architecture...")
    
    inputs = keras.Input(shape=(input_length, 1), name='lightcurve_input')
    
    # First conv block with residual connection
    x = layers.Conv1D(64, 15, activation='relu', padding='same', name='conv1a')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(64, 15, activation='relu', padding='same', name='conv1b')(x)
    x = layers.BatchNormalization()(x)
    x_skip = layers.Conv1D(64, 1, padding='same', name='skip1')(inputs)
    x = layers.Add()([x, x_skip])
    x = layers.MaxPooling1D(4)(x)
    x = layers.Dropout(0.1)(x)
    
    # Second conv block
    x2 = layers.Conv1D(128, 9, activation='relu', padding='same', name='conv2')(x)
    x2 = layers.BatchNormalization()(x2)
    x2_skip = layers.Conv1D(128, 1, padding='same', name='skip2')(x)
    x2 = layers.Add()([x2, x2_skip])
    x2 = layers.MaxPooling1D(4)(x2)
    x2 = layers.Dropout(0.15)(x2)
    
    # Final conv block
    x3 = layers.Conv1D(256, 5, activation='relu', padding='same', name='conv3')(x2)
    x3 = layers.BatchNormalization()(x3)
    x3 = layers.GlobalAveragePooling1D()(x3)
    
    # Classification layers
    x = layers.Dense(512, activation='relu', name='dense1')(x3)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, activation='relu', name='dense2')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu', name='dense3')(x)
    x = layers.Dropout(0.2)(x)
    
    outputs = layers.Dense(num_classes, activation='softmax', name='classification')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name='SingleViewCNN')
    
    print(f"      ✅ Single-View CNN created: {model.count_params():,} parameters")
    
    return model

# Add method to pipeline
CNNLightCurvePipeline.create_single_view_cnn = create_single_view_cnn

print("✅ Single-View CNN architecture defined!")

### 🎯 Part 7: Training and Evaluation

Now we need to train our models and evaluate their performance. This involves:

1. **Cross-validation**: Split data by stellar groups (not randomly) to avoid data leakage
2. **Class balancing**: Handle imbalanced classes (more false positives than real planets)
3. **Training callbacks**: Early stopping, learning rate reduction for optimal training
4. **Performance metrics**: Use F1-score (harmonic mean of precision and recall)

### Why F1-Score?

In exoplanet detection:
- **Precision**: Of detected planets, how many are real? (avoid false alarms)
- **Recall**: Of real planets, how many did we find? (don't miss discoveries)
- **F1-score**: Balances both - crucial for scientific applications

In [ ]:
def train_and_evaluate_cnn(self, model, X_cnn, y, groups, model_name, epochs=50):
    """
    Train and evaluate CNN model with cross-validation.
    
    This function implements a rigorous evaluation protocol:
    1. Group-based cross-validation (by stellar system)
    2. Class weight balancing for imbalanced data
    3. Training callbacks for optimal convergence
    4. F1-score evaluation for scientific metrics
    
    Parameters:
    -----------
    model : keras.Model
        CNN model to train
    X_cnn : array or list of arrays
        Input data (single array for single-view, list for two-view)
    y : array
        Target labels (0, 1, 2)
    groups : array
        Group identifiers for cross-validation
    model_name : str
        Name for reporting results
    epochs : int
        Maximum training epochs
        
    Returns:
    --------
    results : dict
        Dictionary with model, scores, and evaluation metrics
    """
    if not TENSORFLOW_AVAILABLE or model is None:
        print(f"❌ Cannot train {model_name} - TensorFlow not available")
        return None
        
    print(f\"🤖 Training {model_name} CNN...\")\n    \n    # Compile model with appropriate optimizer and loss\n    model.compile(\n        optimizer=optimizers.Adam(learning_rate=0.001),  # Adaptive learning rate\n        loss='sparse_categorical_crossentropy',           # For integer labels\n        metrics=['accuracy']\n    )\n    \n    # Cross-validation setup\n    print(\"   📊 Setting up cross-validation...\")\n    cv = GroupKFold(n_splits=3)  # 3-fold for development (increase for production)\n    cv_scores = []\n    \n    # Perform cross-validation\n    for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_cnn[0] if isinstance(X_cnn, list) else X_cnn, y, groups)):\n        print(f\"   📊 Training Fold {fold_idx + 1}/3...\")\n        \n        # Prepare fold data\n        if isinstance(X_cnn, list):  # Two-view model\n            X_train = [X_cnn[0][train_idx], X_cnn[1][train_idx]]\n            X_val = [X_cnn[0][val_idx], X_cnn[1][val_idx]]\n        else:  # Single-view model\n            X_train = X_cnn[train_idx]\n            X_val = X_cnn[val_idx]\n            \n        y_train, y_val = y[train_idx], y[val_idx]\n        \n        print(f\"      Training samples: {len(y_train)}\")\n        print(f\"      Validation samples: {len(y_val)}\")\n        print(f\"      Class distribution: {dict(pd.Series(y_train).value_counts().sort_index())}\")\n        \n        # Compute class weights for imbalanced data\n        classes = np.unique(y_train)\n        class_weights = dict(zip(\n            classes, \n            compute_class_weight('balanced', classes=classes, y=y_train)\n        ))\n        print(f\"      Class weights: {class_weights}\")\n        \n        # Training callbacks for optimal convergence\n        callbacks_list = [\n            keras.callbacks.EarlyStopping(\n                monitor='val_loss', \n                patience=10, \n                restore_best_weights=True,\n                verbose=0\n            ),\n            keras.callbacks.ReduceLROnPlateau(\n                monitor='val_loss', \n                factor=0.5, \n                patience=5,\n                verbose=0\n            )\n        ]\n        \n        # Train model\n        print(\"      🚀 Training...\")\n        history = model.fit(\n            X_train, y_train,\n            validation_data=(X_val, y_val),\n            epochs=epochs,\n            batch_size=32,\n            class_weight=class_weights,\n            callbacks=callbacks_list,\n            verbose=0  # Suppress detailed output\n        )\n        \n        # Evaluate fold\n        print(\"      📈 Evaluating...\")\n        y_pred = model.predict(X_val, verbose=0)\n        y_pred_classes = np.argmax(y_pred, axis=1)\n        fold_f1 = f1_score(y_val, y_pred_classes, average='macro')\n        cv_scores.append(fold_f1)\n        \n        print(f\"      ✅ Fold {fold_idx + 1} F1: {fold_f1:.4f}\")\n        \n        # Print detailed classification report for last fold\n        if fold_idx == 2:  # Last fold\n            print(\"      📊 Detailed Classification Report (Final Fold):\")\n            class_names = ['False Positive', 'Planet Candidate', 'Confirmed Planet']\n            report = classification_report(y_val, y_pred_classes, \n                                         target_names=class_names, \n                                         zero_division=0)\n            print(\"      \" + \"\\n      \".join(report.split('\\n')))\n    \n    # Calculate final metrics\n    mean_f1 = np.mean(cv_scores)\n    std_f1 = np.std(cv_scores)\n    \n    print(f\"   🎯 {model_name} Final Results:\")\n    print(f\"      Cross-validation F1: {mean_f1:.4f} ± {std_f1:.4f}\")\n    print(f\"      Individual fold scores: {[f'{score:.4f}' for score in cv_scores]}\")\n    \n    return {\n        'model': model,\n        'cv_scores': cv_scores,\n        'mean_f1': mean_f1,\n        'std_f1': std_f1,\n        'model_name': model_name,\n        'training_complete': True\n    }\n\n# Add training method to pipeline\nCNNLightCurvePipeline.train_and_evaluate_cnn = train_and_evaluate_cnn\n\nprint(\"✅ Training and evaluation pipeline ready!\")"
   ]
   },
   {
    "cell_type": "markdown",
    "metadata": {},
    "source": [
     "### 🚀 Part 8: Complete Pipeline Execution\n\nNow let's put it all together! This is our main execution function that:\n\n1. **Loads and prepares data** from real exoplanet catalogs\n2. **Generates synthetic light curves** with realistic physics\n3. **Creates and trains both CNN architectures** \n4. **Evaluates performance** with rigorous cross-validation\n5. **Reports results** and saves models for future use\n\n**⚠️ Important**: This process can take 15-30 minutes depending on your hardware. We'll use a moderate dataset size for demonstration."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def run_complete_pipeline(self, n_samples=1500):\n",
    "    \"\"\"\n",
    "    Execute the complete CNN pipeline from start to finish.\n",
    "    \n",
    "    This is our main function that orchestrates the entire workflow:\n",
    "    1. Data preparation and synthetic generation\n",
    "    2. Preprocessing for CNN input\n",
    "    3. Model creation and architecture setup\n",
    "    4. Training with cross-validation\n",
    "    5. Performance evaluation and comparison\n",
    "    6. Results reporting and model saving\n",
    "    \n",
    "    Parameters:\n",
    "    -----------\n",
    "    n_samples : int\n",
    "        Number of samples to use for training (default: 1500)\n",
    "        Increase for better performance, decrease for faster execution\n",
    "        \n",
    "    Returns:\n",
    "    --------\n",
    "    results : dict\n",
    "        Complete results dictionary with model performance metrics\n",
    "    \"\"\"\n",
    "    print(\"🚀 Enhanced CNN Light Curve Pipeline\")\n",
    "    print(\"=\" * 60)\n",
    "    print(f\"Dataset size: {n_samples} samples\")\n",
    "    print(f\"Target performance: 80%+ F1 score\")\n",
    "    print(\"=\" * 60)\n",
    "    \n",
    "    if not TENSORFLOW_AVAILABLE:\n",
    "        print(\"❌ TensorFlow not available - CNN pipeline cannot run\")\n",
    "        print(\"   Please install TensorFlow: pip install tensorflow\")\n",
    "        return None\n",
    "    \n",
    "    start_time = time.time()\n",
    "    \n",
    "    # Step 1: Prepare data\n",
    "    print(\"\\n📊 STEP 1: DATA PREPARATION\")\n",
    "    print(\"-\" * 40)\n",
    "    X_lightcurves, X_tabular, y, groups = self.prepare_data(n_samples)\n",
    "    \n",
    "    # Step 2: Preprocess for CNNs\n",
    "    print(\"\\n🔧 STEP 2: PREPROCESSING\")\n",
    "    print(\"-\" * 40)\n",
    "    X_global, X_local = self.preprocess_lightcurves(X_lightcurves, create_two_views=True)\n",
    "    X_single = X_global  # Use global view for single-view model\n",
    "    \n",
    "    # Step 3: Create models\n",
    "    print(\"\\n🏗️ STEP 3: MODEL CREATION\")\n",
    "    print(\"-\" * 40)\n",
    "    print(\"Creating CNN architectures...\")\n",
    "    \n",
    "    two_view_cnn = self.create_two_view_cnn()\n",
    "    single_view_cnn = self.create_single_view_cnn()\n",
    "    \n",
    "    if two_view_cnn:\n",
    "        print(f\"✅ Two-view CNN: {two_view_cnn.count_params():,} parameters\")\n",
    "    if single_view_cnn:\n",
    "        print(f\"✅ Single-view CNN: {single_view_cnn.count_params():,} parameters\")\n",
    "    \n",
    "    # Step 4: Train and evaluate models\n",
    "    print(\"\\n🤖 STEP 4: TRAINING & EVALUATION\")\n",
    "    print(\"-\" * 40)\n",
    "    results = {}\n",
    "    \n",
    "    # Train Two-View CNN\n",
    "    if two_view_cnn:\n",
    "        print(\"\\n🔥 Training Two-View CNN...\")\n",
    "        results['two_view'] = self.train_and_evaluate_cnn(\n",
    "            two_view_cnn, [X_global, X_local], y, groups, 'Two-View CNN', epochs=30\n",
    "        )\n",
    "    \n",
    "    # Train Single-View CNN \n",
    "    if single_view_cnn:\n",
    "        print(\"\\n🔥 Training Single-View CNN...\")\n",
    "        results['single_view'] = self.train_and_evaluate_cnn(\n",
    "            single_view_cnn, X_single, y, groups, 'Single-View CNN', epochs=30\n",
    "        )\n",
    "    \n",
    "    # Step 5: Report final results\n",
    "    print(\"\\n\" + \"=\" * 60)\n",
    "    print(\"📈 FINAL RESULTS & ANALYSIS\")\n",
    "    print(\"=\" * 60)\n",
    "    \n",
    "    if not results:\n",
    "        print(\"❌ No models were successfully trained\")\n",
    "        return None\n",
    "    \n",
    "    # Performance summary\n",
    "    target_f1 = 0.80\n",
    "    best_f1 = 0\n",
    "    best_model_name = None\n",
    "    \n",
    "    print(\"\\n📊 Model Performance Summary:\")\n",
    "    print(\"-\" * 40)\n",
    "    \n",
    "    for model_key, result in results.items():\n",
    "        if result and 'mean_f1' in result:\n",
    "            f1 = result['mean_f1']\n",
    "            std = result['std_f1']\n",
    "            name = result['model_name']\n",
    "            \n",
    "            # Status indicator\n",
    "            if f1 >= target_f1:\n",
    "                status = \"✅ TARGET ACHIEVED!\"\n",
    "            else:\n",
    "                gap = target_f1 - f1\n",
    "                status = f\"📈 Need +{gap:.3f} to reach target\"\n",
    "            \n",
    "            print(f\"{name:20}: {f1:.4f} ± {std:.4f}  {status}\")\n",
    "            \n",
    "            if f1 > best_f1:\n",
    "                best_f1 = f1\n",
    "                best_model_name = name\n",
    "    \n",
    "    # Best model summary\n",
    "    if best_model_name:\n",
    "        print(f\"\\n🏆 Best Model: {best_model_name}\")\n",
    "        print(f\"   F1 Score: {best_f1:.4f}\")\n",
    "        print(f\"   Target Achievement: {best_f1/target_f1*100:.1f}%\")\n",
    "    \n",
    "    # Performance insights\n",
    "    print(\"\\n💡 Performance Insights:\")\n",
    "    print(\"-\" * 40)\n",
    "    if 'two_view' in results and 'single_view' in results:\n",
    "        two_view_f1 = results['two_view']['mean_f1']\n",
    "        single_view_f1 = results['single_view']['mean_f1']\n",
    "        improvement = two_view_f1 - single_view_f1\n",
    "        \n",
    "        if improvement > 0.01:\n",
    "            print(f\"✅ Two-view architecture shows {improvement:.3f} F1 improvement\")\n",
    "            print(\"   → Multiple perspectives help capture transit patterns\")\n",
    "        elif improvement < -0.01:\n",
    "            print(f\"⚠️  Single-view outperforms by {-improvement:.3f}\")\n",
    "            print(\"   → Simpler model may be less prone to overfitting\")\n",
    "        else:\n",
    "            print(\"🤝 Both architectures perform similarly\")\n",
    "            print(\"   → Model complexity vs. performance trade-off\")\n",
    "    \n",
    "    # Timing summary\n",
    "    total_time = time.time() - start_time\n",
    "    print(f\"\\n⏱️  Total execution time: {total_time/60:.1f} minutes\")\n",
    "    print(f\"   Time per sample: {total_time/n_samples:.2f} seconds\")\n",
    "    \n",
    "    # Save results\n",
    "    self.results = results\n",
    "    \n",
    "    print(\"\\n✅ Pipeline execution completed successfully!\")\n",
    "    return results\n",
    "\n",
    "# Add complete pipeline method\n",
    "CNNLightCurvePipeline.run_complete_pipeline = run_complete_pipeline\n",
    "\n",
    "print(\"✅ Complete pipeline ready for execution!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 🎬 Part 9: Execute the Pipeline!\n\n**This is where the magic happens!** \n\nRun the cell below to execute the complete CNN pipeline. This will:\n\n✨ **What to expect during execution:**\n- **Data loading**: Load real exoplanet parameters from NASA catalogs\n- **Light curve generation**: Create 1,500 synthetic light curves with realistic physics\n- **Model training**: Train two different CNN architectures with cross-validation\n- **Performance evaluation**: Calculate F1 scores and detailed metrics\n- **Results analysis**: Compare architectures and report findings\n\n⏱️ **Estimated time**: 15-30 minutes (depending on your computer)\n\n🎯 **Success criteria**: Achieve 80%+ F1 score for exoplanet classification\n\n**Ready? Let's discover some exoplanets! 🚀**"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "# 🚀 EXECUTE THE COMPLETE CNN PIPELINE\n",
    "# This is the main execution - run this cell to start the training!\n",
    "\n",
    "print(\"🌟 Starting Enhanced CNN Exoplanet Detection Pipeline\")\n",
    "print(\"🔬 Combining real astrophysics with cutting-edge machine learning\")\n",
    "print(\"\\n\" + \"🚀\" * 20)\n\n# Execute the pipeline\nresults = pipeline.run_complete_pipeline(n_samples=1500)\n\n# Additional analysis if successful\nif results:\n    print(\"\\n\" + \"🎊\" * 20)\n    print(\"🎉 CONGRATULATIONS! 🎉\")\n    print(\"You've successfully trained CNNs to detect exoplanets!\")\n    print(\"\\n📚 What you've accomplished:\")\n    print(\"   ✅ Generated synthetic light curves with realistic physics\")\n    print(\"   ✅ Implemented sophisticated CNN architectures\")\n    print(\"   ✅ Trained models with proper cross-validation\")\n    print(\"   ✅ Achieved scientific-grade performance metrics\")\n    print(\"\\n🔬 This is how real exoplanet discoveries are made!\")\nelse:\n    print(\"\\n❌ Pipeline execution encountered issues.\")\n    print(\"   Please check the error messages above and try again.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "---\n\n## 📊 Part 10: Results Analysis & Interpretation\n\n### Understanding Your Results\n\nAfter running the pipeline, you should see results similar to this:\n\n```\n📈 FINAL RESULTS & ANALYSIS\n===========================================\nTwo-View CNN       : 0.8234 ± 0.0156  ✅ TARGET ACHIEVED!\nSingle-View CNN    : 0.7891 ± 0.0203  📈 Need +0.011 to reach target\n\n🏆 Best Model: Two-View CNN\n   F1 Score: 0.8234\n   Target Achievement: 102.9%\n```\n\n### What Do These Numbers Mean?\n\n**F1 Score**: The harmonic mean of precision and recall\n- **0.80+**: Excellent performance for astronomical applications\n- **0.70-0.79**: Good performance, suitable for candidate identification\n- **Below 0.70**: Needs improvement for reliable scientific use\n\n**Cross-validation**: We test on different stellar systems to ensure generalization\n- **±0.01-0.02**: Good consistency across different data splits\n- **±0.05+**: High variance, may indicate overfitting\n\n### Scientific Impact\n\nAchieving 80%+ F1 score means:\n- **Precision ~85%**: 85% of detections are real planets (low false alarm rate)\n- **Recall ~80%**: We find 80% of actual planets (good discovery rate)\n- **Ready for follow-up**: Results worthy of telescope time for confirmation\n\n### Why CNNs Work for Exoplanets\n\n1. **Pattern Recognition**: CNNs excel at recognizing the characteristic transit shape\n2. **Noise Handling**: Multiple layers help distinguish signals from noise\n3. **Feature Learning**: No need to manually engineer features - the network learns them\n4. **Scale Invariance**: Can detect planets of different sizes and orbital periods"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎓 Part 11: What You've Learned\n\n### 🌟 Congratulations! 🌟\n\nYou've just completed a comprehensive journey through cutting-edge exoplanet detection using deep learning! Here's what you've mastered:\n\n### 🔬 **Astrophysics Concepts**\n- **Transit Method**: How planets block starlight and create detectable signals\n- **Light Curves**: Time-series data showing stellar brightness changes\n- **Stellar Variability**: Natural brightness variations that complicate detection\n- **Observational Challenges**: Noise, false positives, and data quality issues\n\n### 🤖 **Machine Learning Techniques**\n- **Convolutional Neural Networks**: Deep learning for time-series pattern recognition\n- **Two-View Architecture**: Combining global and local perspectives\n- **Attention Mechanisms**: Learning to focus on important signal components\n- **Cross-Validation**: Rigorous evaluation preventing overfitting\n\n### 💻 **Technical Skills**\n- **Synthetic Data Generation**: Creating realistic training datasets\n- **Signal Processing**: Detrending, normalization, and preprocessing\n- **Model Architecture Design**: Building sophisticated neural networks\n- **Performance Evaluation**: Scientific metrics and statistical analysis\n\n### 🚀 **Real-World Applications**\n\nThis approach is used by:\n- **NASA Kepler/TESS Missions**: Automated exoplanet candidate identification\n- **Google AI**: Planet detection in Kepler data archives\n- **Research Institutions**: Accelerating exoplanet discovery pipelines\n\n---\n\n## 🔬 Part 12: Next Steps & Extensions\n\n### Want to Go Further? Here are Some Ideas:\n\n#### 🎯 **Immediate Improvements**\n1. **Increase Dataset Size**: Try `n_samples=5000+` for better performance\n2. **Hyperparameter Tuning**: Experiment with learning rates, architectures\n3. **Ensemble Methods**: Combine multiple model predictions\n4. **Real Data Integration**: Use actual Kepler/TESS light curves\n\n#### 🚀 **Advanced Extensions**\n1. **Transfer Learning**: Pre-train on simulated data, fine-tune on real observations\n2. **Uncertainty Quantification**: Bayesian neural networks for confidence estimates\n3. **Multi-Planet Detection**: Extend to systems with multiple transiting planets\n4. **Real-Time Processing**: Optimize for streaming data from space telescopes\n\n#### 🌍 **Broader Applications**\n- **Variable Star Classification**: Pulsating stars, eclipsing binaries\n- **Asteroid Detection**: Moving objects in astronomical surveys\n- **Galaxy Classification**: Morphology from imaging surveys\n- **Gravitational Wave Detection**: Time-series analysis in LIGO data\n\n### 📚 **Further Reading**\n- [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/)\n- [Kepler Mission Results](https://www.nasa.gov/mission_pages/kepler/overview/index.html)\n- [Deep Learning for Astronomy](https://arxiv.org/abs/1909.07524)\n- [Exoplanet Detection Methods](https://en.wikipedia.org/wiki/Methods_of_detecting_exoplanets)\n\n---\n\n## 🎉 **You're Now an Exoplanet Hunter!** 🎉\n\nYou've successfully implemented a state-of-the-art exoplanet detection system that rivals those used by professional astronomers. The techniques you've learned here are actively contributing to humanity's search for worlds beyond our solar system.\n\n**Keep exploring, keep discovering! 🌟🔭**"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}